In [1]:
import string


# ascii文字で辞書を作る
# string.printable => '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
all_chars = string.printable
vocab_size = len(all_chars)
vocab_dict = dict((c, i) for (i, c) in enumerate(all_chars))

# 文字列を一文字ずつに分解し数値のリストに変換する関数
def str2ints(s, vocab_dict):
    return [vocab_dict[c] for c in s]

# 数値のリストを文字列に変換する関数
def ints2str(x, vocab_array):
    return "".join([vocab_array[i] for i in x])

In [2]:
from torch.utils.data import Dataset


# 
# curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt > tinyshakespeare.txt
class ShakespeareDataset(Dataset):
    def __init__(self, path, chunk_size=200):
        # ファイルを読み込み、数値のリストに変換する
        data = str2ints(open(path).read().strip(), vocab_dict)
        # LongTensorに変換し、chunk_sizeの分だけ塊にしてsplitする
        data = torch.LongTensor(data).split(chunk_size)
        # 最後のchunkの長さをチェックして余った部分は捨てる
        if len(data[-1]) < chunk_size:
            data = data[:-1]
        self.data = data
        self.n_chunks = len(self.data)
        
    def __len__(self):
        return self.n_chunks
    
    def __getitem__(self, idx):
        return self.data[idx]

In [3]:
import torch
from torch.utils.data import DataLoader


ds = ShakespeareDataset('./tinyshakespeare.txt', chunk_size=200)
# DataLoaderによって(batch_size, step_size)になる
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=4)

In [4]:
from torch import nn


class SequenceGenerationNet(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        # Linearのoutputサイズは最初のEmbeddingのinputサイズと同じnum_embeddings
        # num_embeddings分だけ確率を出力してall_charsから尤もらしい文字を選ぶため
        # linearの出力は(n, n)
        self.linear = nn.Linear(hidden_size, num_embeddings)
        
    def forward(self, x, h0=None):
        # xは(batch_size, step_size)から(batch_size, step_size, embedding_dim)に変換される
        x = self.emb(x)
        # lstmによって、xは入力(batch_size, step_size, embedding_dim)から出力(batch_size, step_size, hidden_size)に変換される
        x, h = self.lstm(x, h0)
        # 線形層により(batch_size, step_size, hidden_size)から(batch_size, step_size, num_embeddings)に変換される
        x = self.linear(x)
        return x, h

In [5]:
def generate_seq(net, start_phrase='The King said', length=200, temperature=0.8):
    # モデルを評価モードにする
    net.eval()
    # 出力の数値を収納するリスト
    result = []
    # 開始文字列をTensorに変換
    start_tensor = torch.LongTensor(str2ints(start_phrase, vocab_dict))
    # 先頭にbatch次元を付けてVariableにする
    x0 = V(start_tensor.unsqueeze(0), volatile=True)
    # RNNに通して出力と新しい内部状態を得る
    o, h = net(x0)
    # バッチごとの、時系列出力の最後の部分を(正規化されてない)確率に変換
    # 正規化しても良いが、torch.multinomialは正規化しなくてもちゃんとサンプリングできるのと、ソフトマックス関数の分子の部分だけでも十分サンプリングできるのであまり意味ない
    out_dist = o[:, -1].data.view(-1).exp()
    # 確率から実際の文字のインデクスをサンプリング
    top_i = torch.multinomial(out_dist, 1)[0]
    # 結果を保存
    result.append(top_i)
    # 生成された結果を次々にRNNに入力していく
    for i in range(length):
        inp = torch.LongTensor([[top_i]])
        o, h = net(V(inp), h)
        out_dist = o.data.view(-1).exp()
        top_i = torch.multinomial(out_dist, 1)[0]
        result.append(top_i)
    # 開始文字列と生成された文字列をまとめて返す
    return start_phrase + ints2str(result, all_chars)

In [6]:
from torch.autograd import Variable as V
from statistics import mean
from torch import optim


net = SequenceGenerationNet(vocab_size, 20, 50, num_layers=2, dropout=0.1)
opt = optim.Adam(net.parameters())
# 多クラス分類なので損失関数はソフトマックスクロスエントロピー
loss_f = nn.CrossEntropyLoss()
for epoch in range(50):
    net.train()
    losses = []
    for data in loader:
        # xは初めから最後の手前の文字列まで
        x = V(data[:, :-1])
        # yは2文字目から最後の文字まで
        y = V(data[:, 1:])
        y_pred, _ = net(x)
        # batchとstepを結合して損失関数を計算
        # y_pred.size()                      => (32, 199, 100)
        # y.size()                           => (32, 199)
        # y_pred.view(-1, vocab_size).size() => (6368, 100)
        # y.contiguous().view(-1).size()     => (6368)
        # contiguous()はメモリにのってないtensorを計算するときに必要
        # https://discuss.pytorch.org/t/runtimeerror-input-is-not-contiguous/930/6
        loss = loss_f(y_pred.view(-1, vocab_size), y.contiguous().view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
    # 現在の損失関数と生成される文章の例を表示
    print('===================================================================================')
    print(epoch, mean(losses))
    print(generate_seq(net))

0 3.4951481083461218
The King saidc oWd|rCondenw
hls
t,a n innddeto yafW nee
 hM,e\dao  Iaw .gI eekc rhhslteb e
n le Rmuto>Tmtohrtpoiteoh oW 
teaosyN
uiivaaeN ohe,v s naui heyon ddvd
 k# teein-stto: N nStw4v;fha Aet waReen Gt,soeihr!Bt
1 3.0123700101034983
The King saidf we aainis agk yathe isn lagmu wisy
Atobr ce atmre tiot 'alliro fr nale iost chdrevS tihy hs thamame lacltn pis hnrsca aiodt oe nah!
yutnanae ageenL
Au goo messer,
ItydC. By: ad.d
I delll hyet,s sytr 
2 2.6087265287126815
The King said!:
ath,Wl
-el suchnlit'n y'or theNU
ske:tctfale
Wule
Uomy whes ad was, mp.e

Ye de,: bory: I heurrbes: whare nes hp ou rabe,lt: ermike
 ouss navele whlar
Yodncattir ue,

YU,N:
 horls bto gteee oL mer 
3 2.4075860813685828
The King said theres sut lafgose het I;
Kt: ti yuner
Hid hohe'gne urbted ere venes, the, ar wencanh thesn vythbely: sase eik gore tis the mavhy lil lit ol cofbile soulsaln amde
Cot thit dast borle nast, h'is
Late p
4 2.3129516070229665
The King said the; I.
Bid whe medori